In [1]:
import sys, os

sys.path.insert(0, "/home/ang/usd_ang/lib/python")
os.environ["LD_LIBRARY_PATH"] = "/home/ang/usd_ang/lib"

import pxr


# SysML → USD Digital Assembly Case Study

This notebook demonstrates the full S24 pipeline:

1. Parse a SysML v2 model
2. Convert it to a structured JSON representation
3. Vet and validate the system graph
4. Generate USD geometry, materials, and components
5. Assemble the full system hierarchy into a USD scene

The output is a USD assembly suitable for:
- Visualization (usdview / Omniverse)
- Simulation coupling
- Digital twin workflows

In [2]:
# Standard library
import os
from pathlib import Path

# S24 pipeline
from S24.sysml import sysml_to_json, write_json
from S24.jsonio.vetting import VettingProc
from S24.usd.builder import USDBuilder


## Project Paths

We explicitly define all paths so the pipeline is reproducible and independent
of the working directory.


In [3]:
ROOT = Path.cwd().parent  # repo root if notebook is in notebooks/

DATA_SYSML = ROOT / "database" / "sysml"
DATA_JSON  = ROOT / "database" / "json"

ASSETS_DIR = ROOT / "database" / "assets"
SCENES_DIR = ROOT / "database" / "scenes"

DATA_SYSML.mkdir(parents=True, exist_ok=True)
DATA_JSON.mkdir(parents=True, exist_ok=True)
SCENES_DIR.mkdir(parents=True, exist_ok=True)

SYSML_FILE = DATA_SYSML / "habitation.sysml"
JSON_FILE  = DATA_JSON  / "habmod.json"
SCENE_FILE = SCENES_DIR / "HabitationAssembly.usda"

SYSML_FILE, JSON_FILE, SCENE_FILE


(PosixPath('/home/ang/Desktop/GC/main/GC-Sysml-OpenUSD/database/sysml/habitation.sysml'),
 PosixPath('/home/ang/Desktop/GC/main/GC-Sysml-OpenUSD/database/json/habmod.json'),
 PosixPath('/home/ang/Desktop/GC/main/GC-Sysml-OpenUSD/database/scenes/HabitationAssembly.usda'))

## Step 1 — Load SysML Model

We load the SysML v2 text model that defines the system structure,
attributes, and hierarchy.


In [4]:
with open(SYSML_FILE, "r", encoding="utf-8") as f:
    sysml_text = f.read()

print(sysml_text[:500], "...")

    package 'Habitation Module' {

        // -------------------------------------------------------------------------
        // Basic items
        // -------------------------------------------------------------------------
        private import SysML::*;
        private import SI::*;
        private import ISQBase::*;
        private import ScalarValues::*;

        part HabitationModule {
            part HabitationModule_dims {
                attribute length = 15; // meters
            ...


## Step 2 — Convert SysML to JSON

The SysML model is converted into a flat JSON list of parts with:
- Explicit parent / children relationships
- Dimensions in SI units
- Evaluated attributes
- Metadata preserved


In [5]:
parts_json = sysml_to_json(
    sysml_text,
    namespace="lunarspaceport1"
)

write_json(parts_json, JSON_FILE)

len(parts_json)

3

In [6]:
parts_json[0]

{'type': 'Part',
 'id': 'urn:lunarspaceport1:part:HabitationModule:001',
 'name': 'HabitationModule',
 'dimensions': {'dims_m': [15.0, 10.0, 5.0],
  'metersPerUnit': 1,
  'upAxis': 'Z',
  'X': 0,
  'Y': 0,
  'Z': 0},
 'attributes': {'HabitationModule_volume_m3': 750.0},
 'metadata': {'geometry': 'assets/geom/HabModule_geom.usda',
  'material': 'assets/mtl/titanium.usda'},
 'children': ['O2Tank1', 'O2Tank2']}

## Step 3 — Vet JSON Representation

We now perform structural and semantic validation:
- Required fields
- Unique part names
- Parent–child consistency
- Cycle detection
- Single connected system graph


In [7]:
vetting = VettingProc(source=str(JSON_FILE))
vetted_parts = vetting.by_name

list(vetted_parts.keys())


['HabitationModule', 'O2Tank1', 'O2Tank2']

## Step 4 — Generate USD Assets

For each vetted part, we generate:
- Geometry layer
- Material layer
- Component layer

These live under `database/assets/`.


In [8]:
builder = USDBuilder(
    vetted_parts,
    overwrite=True,
    use_paths_from_vetted=False
)

outputs = builder.build_all_parts()
outputs


{'HabitationModule': {'geom': 'database/assets/geoms/HabitationModule_geom.usda',
  'mat': 'database/assets/mats/HabitationModule_mat.usda',
  'component': 'database/assets/components/HabitationModule.usda'},
 'O2Tank1': {'geom': 'database/assets/geoms/O2Tank1_geom.usda',
  'mat': 'database/assets/mats/O2Tank1_mat.usda',
  'component': 'database/assets/components/O2Tank1.usda'},
 'O2Tank2': {'geom': 'database/assets/geoms/O2Tank2_geom.usda',
  'mat': 'database/assets/mats/O2Tank2_mat.usda',
  'component': 'database/assets/components/O2Tank2.usda'}}

## Step 5 — Assemble Full USD Scene

We instantiate the vetted hierarchy into a single USD stage
using component references and transforms.


In [9]:
scene_path = builder.write_assembly_scene(
    root_name="HabitationModule",
    include_root_as_instance=True,
    instanceable=False,
    debug_refs=True
)

scene_path

[REF] /World/HabitationModule -> ../assets/components/HabitationModule.usda :/HabitationModule
[REF] /World/HabitationModule/O2Tank1 -> ../assets/components/O2Tank1.usda :/O2Tank1
[REF] /World/HabitationModule/O2Tank2 -> ../assets/components/O2Tank2.usda :/O2Tank2


'database/scenes/assembly.usda'

## Step 6 — View the Result

Open the generated assembly in usdview:

```bash
usdview database/scenes/HabitationAssembly.usda
